Ceneo Scraper 

## Struktura opinii w serwisie Ceneo.pl

|skadowa|zmienna|sekeltor|
|------|-------|--------|
|identyfikator opinii|opinion_id|["data-entry-id"]|
|autora|author|span.user-post__author-name|
|rekomendację|recomendation|span.user-post__author-recomendation > em|
|liczbę gwiazdek|rating|span.user-post__score-count|
|treść opinii|content|div.user-post__text|
|listę zalet|pros|div.review-feature__title--positives ~ div.review-feature__item|
|listę wad|cons|div.review-feature__title--negatives ~ div.review-feature__item|
|data wystawienia opinii|opinion_date|span.user-post__published > time:nth-child(1)["datetime"]|
|data zakupu produktu|purchase_date|opinion_date|span.user-post__published > time:nth-child(2)["datetime"]|
|ile osób uznało opinię za przydatną|likes|button.vote-yes > span|
|ile osób uznało opinię za nieprzydatną|dislikes|button.vote-no > span|

## Import bibliotek 

In [34]:
import os
import json
import requests 
from bs4 import BeautifulSoup
from flask import Flask, request

Funkcja do ekstrakcji zawartości ze strony HTML

In [35]:
def extract(ancestor,selector, attribute = None, return_list = False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.get_text().strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.get_text().strip()

## strona produktu


In [36]:
product_id = input("Podaj kod produktu z ceneo.pl: ")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"


## wysyłanie zadania dostepu do zasobu

In [37]:
response = requests.get(url)
response.status_code


200

In [38]:
selectors = {
'author': ("span.user-post__author-name",),
'recomendation' : ("span.user-post__author-recomendation > em",),
'rating': ("span.user-post__score-count",),
'content': ("div.user-post__text",),
'pros': ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
'cons': ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
'opinion_date': ("span.user-post__published > time:nth-child(1)","datetime"),
'purchase_date': ("opinion_date|span.user-post__published > time:nth-child(2)","datetime"),
'likes': ("button.vote-yes > span",),
'dislikes': ("button.vote-no > span",),
'opinion_id': (None,"data-entry-id"),
}



In [39]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")

In [40]:
all_opinions = []
while(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select('div.js_product-review')
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)

        print(single_opinion)
    try:
        url = "https://www.ceneo.pl" + extract(page, "a.pagination__next", "href")
    except TypeError:
        break

{'author': 'm...z', 'recomendation': 'Polecam', 'rating': '5/5', 'content': 'Co tu oceniań, produkt absolutnie be zarzutu!', 'pros': ['dokładny touchpad', 'jakość', 'wydajność', 'wygląd', 'wygodna klawiatura'], 'cons': ['mało portów', 'może mały dysk'], 'opinion_date': '2023-01-26 20:12:02', 'purchase_date': None, 'likes': '1', 'dislikes': '0', 'opinion_id': '17067606'}
{'author': 'a...i', 'recomendation': 'Polecam', 'rating': '5/5', 'content': 'Wiedziałem, że będzie dobrze ale jest jeszcze lepiej.', 'pros': [], 'cons': [], 'opinion_date': '2024-05-15 23:17:42', 'purchase_date': None, 'likes': '0', 'dislikes': '1', 'opinion_id': '18651225'}
{'author': 'b...a', 'recomendation': 'Polecam', 'rating': '5/5', 'content': 'idealny do pracy w biurze.', 'pros': [], 'cons': [], 'opinion_date': '2024-04-01 12:05:03', 'purchase_date': None, 'likes': '0', 'dislikes': '0', 'opinion_id': '18539334'}
{'author': 'l...a', 'recomendation': 'Polecam', 'rating': '5/5', 'content': 'Polecam', 'pros': [], 'co

In [41]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
with open(f"opinions/{product_id}.json","w",encoding="UTF-8") as jf:
    json.dump(all_opinions,jf,indent=4,ensure_ascii=False)